In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import random

# Step 2: Get stock info from Finviz
def get_finviz_stock_info(ticker):
    url = f"https://finviz.com/quote.ashx?t={ticker}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Company name: Found in the quote-header_ticker-wrapper_company class
        name_element = soup.find("h2", class_="quote-header_ticker-wrapper_company")
        name = name_element.text.strip() if name_element else "N/A"
        
        # Sector and Industry: Found in the quote-links section
        sector = "N/A"
        industry = "N/A"
        
        # Look for the links in the first table row that contains sector and industry
        quote_links_div = soup.find("div", class_="quote-links")
        if quote_links_div:
            links = quote_links_div.find_all("a")
            
            # Iterate through the links
            for link in links:
                href = link.get('href', '')
                if 'v=111&f=sec_' in href:
                    sector = link.text.strip()
                elif 'v=111&f=ind_' in href:
                    industry = link.text.strip()
        
        # Find the snapshot table for other data
        data = {}
        snapshot_table = soup.find("table", class_="snapshot-table2")
        if snapshot_table:
            rows = snapshot_table.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                for i in range(0, len(cells), 2):
                    if i + 1 < len(cells):
                        key = cells[i].text.strip()
                        value = cells[i + 1].text.strip()
                        data[key] = value
        
        # Extract price, change, market cap
        price = 0.0
        change = 0.0
        market_cap = "N/A"
        
        # Try to get price from the quote-price section
        price_element = soup.find("strong", class_="quote-price_wrapper_price")
        if price_element:
            try:
                price = float(price_element.text.strip().replace(",", ""))
            except:
                pass
        
        # Try to get change from the snapshot table
        if 'Change' in data:
            try:
                # Remove % and convert to float
                change_str = data['Change'].replace('%', '').replace(',', '').strip()
                change = float(change_str)
            except:
                change = 0.0


        
        # Try to get market cap from the snapshot table
        if 'Market Cap' in data:
            market_cap = data['Market Cap']
        
        return {
            "ticker": ticker,
            "name": name,
            "price": round(price, 2),
            "change": round(change, 2),
            "marketCap": market_cap,
            "sector": sector,
            "industry": industry
        }

    except Exception as e:
        print(f"Error fetching {ticker}: {e}")
        return {
            "ticker": ticker,
            "name": "N/A",
            "price": 0.0,
            "change": 0.0,
            "marketCap": "N/A",
            "sector": "N/A",
            "industry": "N/A"
        }

# Step 3: Run on 20 most popular stocks
all_data = []

# List of 20 most popular/significant stocks (based on market cap, volume, and general interest)
popular_tickers = [
    "AAPL",   # Apple
    "MSFT",   # Microsoft
    "AMZN",   # Amazon
    "GOOGL",  # Alphabet (Google) Class A
    "META",   # Meta Platforms (Facebook)
    "TSLA",   # Tesla
    "JPM",    # JPMorgan Chase
    "V",      # Visa
    "BAC",    # Bank of America
    "DIS",    # Walt Disney
    "NFLX",   # Netflix
    "AMD",    # Advanced Micro Devices
    "KO",     # Coca-Cola
    "PFE",    # Pfizer
    "INTC",   # Intel
    "NKE",    # Nike
]

for ticker in popular_tickers:
    info = get_finviz_stock_info(ticker)
    all_data.append(info)
    print(f"Processed {ticker}: {info}")
    # Random delay between 1.5 and 2.5 seconds to avoid rate limiting
    time.sleep(1.5 + random.random())

# Step 4: Show result and save to CSV
df = pd.DataFrame(all_data)
print(df)

# Optionally save to CSV
df.to_csv('popular_stocks_data.csv', index=False)

Processed AAPL: {'ticker': 'AAPL', 'name': 'Apple Inc', 'price': 212.33, 'change': 0.0, 'marketCap': '3171.32B', 'sector': 'Technology', 'industry': 'Consumer Electronics'}
Processed MSFT: {'ticker': 'MSFT', 'name': 'Microsoft Corporation', 'price': 452.94, 'change': 0.0, 'marketCap': '3366.50B', 'sector': 'Technology', 'industry': 'Software - Infrastructure'}
Processed AMZN: {'ticker': 'AMZN', 'name': 'Amazon.com Inc', 'price': 210.25, 'change': 0.0, 'marketCap': '2232.09B', 'sector': 'Consumer Cyclical', 'industry': 'Internet Retail'}
Processed NVDA: {'ticker': 'NVDA', 'name': 'NVIDIA Corp', 'price': 135.34, 'change': 0.0, 'marketCap': '3300.61B', 'sector': 'Technology', 'industry': 'Semiconductors'}
Processed GOOGL: {'ticker': 'GOOGL', 'name': 'Alphabet Inc', 'price': 165.37, 'change': 0.0, 'marketCap': '2014.46B', 'sector': 'Communication Services', 'industry': 'Internet Content & Information'}
Processed META: {'ticker': 'META', 'name': 'Meta Platforms Inc', 'price': 659.36, 'chang

In [2]:
from services.market_service import fetch_market_data

In [3]:
fetch_market_data()

[{'ticker': 'AAPL',
  'name': 'Apple Inc',
  'price': 212.33,
  'change': 0.0,
  'marketCap': '3171.32B',
  'sector': 'Technology',
  'industry': 'Consumer Electronics'},
 {'ticker': 'MSFT',
  'name': 'Microsoft Corporation',
  'price': 452.94,
  'change': 0.0,
  'marketCap': '3366.50B',
  'sector': 'Technology',
  'industry': 'Software - Infrastructure'},
 {'ticker': 'AMZN',
  'name': 'Amazon.com Inc',
  'price': 210.25,
  'change': 0.0,
  'marketCap': '2232.09B',
  'sector': 'Consumer Cyclical',
  'industry': 'Internet Retail'},
 {'ticker': 'NVDA',
  'name': 'NVIDIA Corp',
  'price': 135.34,
  'change': 0.0,
  'marketCap': '3300.61B',
  'sector': 'Technology',
  'industry': 'Semiconductors'},
 {'ticker': 'GOOGL',
  'name': 'Alphabet Inc',
  'price': 165.37,
  'change': 0.0,
  'marketCap': '2014.46B',
  'sector': 'Communication Services',
  'industry': 'Internet Content & Information'},
 {'ticker': 'META',
  'name': 'Meta Platforms Inc',
  'price': 659.36,
  'change': 0.0,
  'marketCa